In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
window_size = 60
Npi = 100
bandwidth = 0.1
all_prices = []
for ticker in tickers:
    file_path = f"{folder}{ticker}.csv"
    df = pd.read_csv(file_path, parse_dates=['Date'])
    price_col = 'Adjusted Close' if 'Adjusted Close' in df.columns else 'Adj Close'
    df = df[['Date', price_col]].rename(columns={price_col: ticker})
    all_prices.append(df)

data = all_prices[0]
for df in all_prices[1:]:
    data = data.merge(df, on='Date', how='inner')
data = data.set_index('Date')
data = data.loc["2011-06-01":"2021-05-31"]
log_returns = np.log(data / data.shift(1)).dropna()

def create_sequences(log_returns, window):
    sequences = []
    for i in range(len(log_returns) - window):
        seq = log_returns.iloc[i:i+window].values
        sequences.append(seq)
    return np.array(sequences)

data_series = create_sequences(log_returns, window_size)

def quartic_kernel(x, h):
    norm = np.linalg.norm(x) / h
    return ((1 - norm**2)**2) / h if norm < 1 else 0.0

def Fi(t, xti, x, xtip1, ti, tip1):
    dt1 = tip1 - t
    dt2 = tip1 - ti
    term1 = -np.linalg.norm(xtip1 - x)**2 / (2 * dt1)
    term2 = np.linalg.norm(xtip1 - xti)**2 / (2 * dt2)
    return np.exp(term1 + term2)

def estimate_drift(t, x, xi, data_series, ti, tip1, h):
    num = np.zeros_like(x)
    denom = 0.0
    for sample in data_series:
        if tip1 >= sample.shape[0]:
            continue
        xti = sample[ti]
        xtip1 = sample[tip1]
        Fi_val = Fi(t, xti, x, xtip1, ti, tip1)
        weight = Fi_val
        for j in range(ti):
            weight *= quartic_kernel(xi[j] - sample[j], h)
        num += weight * (xtip1 - x)
        denom += weight
    return (1 / (tip1 - t)) * num / (denom + 1e-8)

def sbts_simulation(data_series, Npi=100, h=0.1):
    M, N, d = data_series.shape
    x = [np.zeros(d)]
    for i in range(N - 1):
        y = x[i].copy()
        for k in range(Npi - 1):
            tpi_k = i + k / Npi
            drift = estimate_drift(tpi_k, y, np.array(x), data_series, ti=i, tip1=i+1, h=h)
            eps = np.random.randn(d)
            y = y + (1 / Npi) * drift + (1 / np.sqrt(Npi)) * eps
        x.append(y)
    return np.array(x)

simulated = sbts_simulation(data_series, Npi=Npi, h=bandwidth)
df_sim = pd.DataFrame(simulated, columns=tickers)
print(df_sim)

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/3356709821.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/3356709821.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/3356709821.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd6

         AAPL      MSFT       GOOG      AMZN       NFLX
0    0.000000  0.000000   0.000000  0.000000   0.000000
1    0.087440  0.125506  -0.042302  0.352117  -0.226170
2   -0.255812  0.044919   0.008706  0.121700   0.094032
3   -0.783828 -0.003085  -1.105920  0.011782   0.509091
4   -0.911085 -1.056540  -0.193329 -1.681196   0.369114
5   -2.806054  0.377044   0.376248 -2.475813   1.178833
6   -3.297673  1.286956   1.467785 -3.154921   0.084284
7   -4.357869  2.642923   3.095697 -2.879970  -2.177233
8   -4.974062  2.732975   4.039297 -2.530258  -2.083271
9   -5.720488  1.821225   4.499258 -3.019570  -3.109992
10  -6.579920  0.657004   4.918139 -3.192141  -2.297582
11  -6.277347  2.394310   4.287204 -3.477482  -0.869326
12  -6.720737  1.600864   6.696922 -3.150128  -0.999744
13  -7.255023  1.031138   5.463316 -3.189757  -1.586871
14  -8.986217  0.206368   6.112729 -4.026451  -1.631746
15 -10.396075 -0.923047   5.555610 -2.922712  -2.941426
16 -11.611194 -1.425409   7.440750 -1.223184  -4

In [4]:
returns = df_sim.diff().dropna()

mu = returns.mean().values
cov = returns.cov().values
inv_cov = np.linalg.inv(cov)
ones = np.ones(len(mu))

w_sharpe = inv_cov @ mu
w_sharpe /= np.sum(w_sharpe)

portfolios = pd.DataFrame([w_sharpe],
                          columns=df_sim.columns,
                          index=['MaxSharpe'])
print(portfolios)

              AAPL     MSFT      GOOG      AMZN     NFLX
MaxSharpe  2.16631  0.42312 -2.019201 -1.475789  1.90556


In [5]:
import os

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
start_date = '2021-06-01'
end_date = '2021-09-01'
initial_investment = 100000  # можешь изменить
weights = np.array([2.16631, 0.42312, -2.019201, -1.475789, 1.90556])

close_prices = pd.DataFrame()

for ticker in tickers:
    filepath = os.path.join(folder, f"{ticker}.csv")
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    close_prices[ticker] = df['Adjusted Close']

close_prices = close_prices.loc[start_date:end_date]

start_prices = close_prices.iloc[0]
end_prices = close_prices.iloc[-1]

shares_bought = (initial_investment * weights) / start_prices

final_value = np.sum(shares_bought * end_prices)

print(f"Initial investment: ${initial_investment:,.2f}")
print(f"Final portfolio value after 2 months: ${final_value:,.2f}")
print(f"Return: {(final_value / initial_investment - 1) * 100:.2f}%")


Initial investment: $100,000.00
Final portfolio value after 2 months: $96,126.57
Return: -3.87%


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/2152170420.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/2152170420.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_75154/2152170420.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['